# 1. Determinación de bases y promedios para tramos

## A. Info a utilizar

* CWTR SUM 
* XML (otro empleador)
* Medidas SAP (IT 0)

## B. Importes consolidados por mes

In [1]:
#importo las librerías que voy a usar

import pandas as pd
from data import CWTR, CWTR_SUM, DOTACION, CUILES
from deducciones.incremental.calculo import *
import empleado as ee

In [2]:
#primero tengo que consolidar la info que traigo de SAP (CWTR_SUM) y el XML (ingresos otro empleador)

#me traigo la info de la CWTR_SUM ya que esta los tiene agrupados por importe por mes
df_rem_mes = pd.DataFrame([], columns=["legajo", "mes", "importe"])
count = 0
for index, row in DOTACION.iterrows():
    
    # count+=1
    # if count == 50:
    #     break
    legajo = row["legajo"]
    
    for m in range(1,13):
        restar_prorrateos = DeduccionEspecial.obtener_rem_mes(legajo,m)
        datos_emp = [[legajo, m, restar_prorrateos]]
        df1 = pd.DataFrame(datos_emp, columns=["legajo", "mes", "importe"])
        df_rem_mes = pd.concat([df_rem_mes, df1], ignore_index=True) #guardar en el mismo df, el concatenado de ese df (inicialmente vacío) y el que voy creando con cada iteración. En la segunda iteración va tener ese df mas el nuevo y asi sucesivamente. 

display(df_rem_mes)

,legajo,mes,importe
0,6000002,1,466593.92
1,6000002,2,1405581.67
2,6000002,3,359751.07
3,6000002,4,363686.19
4,6000002,5,363361.69
...,...,...,...
37171,6109479,8,0.00
37172,6109479,9,0.00
37173,6109479,10,0.00
37174,6109479,11,0.00


In [3]:
#leo el df 572 parquet
from helpers.bases import tabla

df_572 = tabla('572web')
# df_572.to_clipboard()
df_572

,cuil,codigo,importe,mes
0,27959504305,Q-Deduc-1,4380.80,7
1,27959504305,Q-Deduc-2,3962.89,13
2,27959504305,Q-Deduc-22,18000.00,12
3,27958569993,Q-RetPerPagCu-27,113153.25,13
4,27958569993,Q-Deduc-22,8400.00,1
...,...,...,...,...
77554,20115449894,Q-Deduc-1,15992.93,8
77555,20115449894,Q-Deduc-1,11015.22,9
77556,20115449894,Q-Deduc-1,15526.32,10
77557,20115449894,Q-Deduc-1,15606.62,11


In [4]:
# dejo las columnas que necesito
filtro = df_572["codigo"].eq("Q-OE-ganBrut")
df_filtrado = df_572[filtro]
df_filtrado = df_filtrado[["cuil", "mes", "importe"]].reset_index(drop=True)
#paso la columna cuil a int, ya que si no el cruce me da error
df_filtrado["cuil"]=df_filtrado["cuil"].astype("int64")
#agrupo por importe los ingresos de otro empleador
df_base_OE = df_filtrado.groupby(['cuil','mes'])['importe'].sum().reset_index() 
df_base_OE

,cuil,mes,importe
0,20201963096,1,16296.94
1,20201963096,2,17215.09
2,20201963096,3,19842.10
3,20201963096,4,19842.10
4,20201963096,5,19842.10
...,...,...,...
62,27279548375,8,17043.89
63,27279548375,9,17813.89
64,27279548375,10,17923.89
65,27279548375,11,18690.77


In [39]:
#como solo tengo el dato del cuil, curzo con el archivo CUILES y me traigo el legajo
df_oe_unido = df_base_OE.merge(CUILES, how='left', left_on="cuil", right_on="cuil")

print("[!] Se eliminarán siguientes datos por no ser un CUIL a procesar:")
display(df_oe_unido[df_oe_unido['legajo'].isnull()])

df_oe_unido = df_oe_unido[["legajo", "mes", "importe"]].reset_index(drop=True)

#luego de que ya tengo el legajo, concateno con el dataframe de la cwtr y ahora tengo el total de rem por mes
df_rem_mes_finales = pd.concat([df_rem_mes, df_oe_unido], ignore_index=True)

# Se eliminan aquellos registros de empleados que no están en el listado de cuits:
df_rem_mes_finales.dropna(inplace=True)

[!] Se eliminarán siguientes datos por no ser un CUIL a procesar:


,cuil,mes,importe,legajo,nombre
23,20344885797,9,75400.88,NaN,NaN
24,20344885797,10,283012.05,NaN,NaN
25,20344885797,11,283012.05,NaN,NaN
26,20344885797,12,283012.05,NaN,NaN


## C. Cálculo de promedios a cada mes 

In [40]:
df_promedios = pd.DataFrame()
count = 0

for index, row in df_rem_mes_finales.iterrows():
    
    # count+=1
    # if count == 30:
    #     break

    legajo = row["legajo"]
    mes = row["mes"]
    importe = row["importe"]
    
    norma = DeduccionEspecial.norma_aplicable(mes)
    valor_promedio = DeduccionEspecial.obtener_promedio_rem_df(legajo, mes, norma, df_rem_mes_finales)
    
    base_tramo = min(importe,valor_promedio)
    
    tramo = DeduccionEspecial.obtener_tramo(base_tramo, mes)

    datos_emp = [[legajo, mes, importe, valor_promedio, base_tramo, tramo]]
    df1 = pd.DataFrame(datos_emp, columns=["legajo","mes", "importe", "promedio", "base_tramo", "tramo"])
    df_promedios = pd.concat([df_promedios, df1], ignore_index=True) #guardar en el mismo df, el concatenado de ese df (inicialmente vacío) y el que voy creando con cada iteración. En la segunda iteración va tener ese df mas el nuevo y asi sucesivamente. 

display(df_promedios)

,legajo,mes,importe,promedio,base_tramo,tramo
0,6000002.0,1,466593.92,466593.92,466593.92,3
1,6000002.0,2,1405581.67,936087.80,936087.80,3
2,6000002.0,3,359751.07,743975.55,359751.07,3
3,6000002.0,4,363686.19,648903.21,363686.19,3
4,6000002.0,5,363361.69,591794.91,363361.69,3
...,...,...,...,...,...,...
37234,6004104.0,8,17043.89,146229.22,17043.89,1
37235,6004104.0,9,17813.89,153587.43,17813.89,1
37236,6004104.0,10,17923.89,158343.41,17923.89,1
37237,6004104.0,11,18690.77,179506.65,18690.77,1


In [43]:
#lo guardo en un Excel
df_promedios.to_excel("../bases-ganancias-2022/info/bases_por_mes.xlsx")

In [42]:
#se deja la base guardada en middle para despues ser utilizada en otros cálculos 
df_promedios.to_parquet('../bases-ganancias-2022/middle/bases_tramos.parquet', compression='brotli')